In [34]:
%load_ext autoreload
%autoreload 2

In [79]:

import numpy as np
import matplotlib.pyplot as plt
import pickle
import cv2

import numpy as np
from tqdm import tqdm
import os

from core.robot import Robot, body_T_optical
from core.dataloader import EncoderIMULoader, LidarLoader, DisparityLoader, RGBLoader
from core.map import Map
from core.load_data import load_data
import matplotlib.pyplot as plt

In [42]:
N = 10
sigma = 0.01
dataset_num = 20
dataset_dir = "data"
data = load_data(dataset_dir, dataset_num)

save_dir = f"results/temp"
robot = Robot(N = N, sigma = sigma, parallize=False, dataset_dir=save_dir)

encoder = EncoderIMULoader(data)
lidar_data = LidarLoader(data)

encoder_stamps = encoder.encoder_stamps
lidar_stamps = lidar_data.lidar_stamps

timestamps = np.concatenate((encoder_stamps, lidar_stamps))
timestamps.sort()

for index, timestamp in tqdm(enumerate(timestamps)):
    if timestamp in encoder_stamps:
        encoder_index = np.where(encoder_stamps == timestamp)[0][0]

        if encoder_index == 0:
            continue

        motion_model = encoder.motion_model(encoder_index)
        robot.predict(motion_model)

    elif timestamp in lidar_stamps:
        lidar_index = np.where(lidar_stamps == timestamp)[0][0]
        lidar_points = lidar_data(lidar_index)

        robot.update(lidar_points.points)
    else:
        print("Something went wrong")

9918it [05:28, 30.22it/s]


In [49]:
from core.utils import coord3d

In [77]:
disp_path = "data/dataRGBD/Disparity20/"
rgb_path = "data/dataRGBD/RGB20/"

In [51]:
lidar_stamps = lidar_data.lidar_stamps
robot_poses = robot.state_history[1:, :3]
assert lidar_stamps.shape[0] == robot_poses.shape[0]

disparity_stamps = DisparityLoader(data).disp_stamps
rgb_stamps = RGBLoader(data).rgb_stamps

In [61]:
def sync_data(original_data, original_stamps, target_stamps):
    index = np.argmin(np.abs(original_stamps - target_stamps))
    return original_data[index]

In [70]:
import glob

len(glob.glob(disp_path + "*.png")), len(glob.glob(rgb_path + "*.png"))
assert len(glob.glob(disp_path + "*.png")) == disparity_stamps.shape[0]
assert len(glob.glob(rgb_path + "*.png")) == rgb_stamps.shape[0]

In [94]:

for disp_index, disparity_stamp in enumerate(disparity_stamps):

    rgb_stamp = sync_data(rgb_stamps, disparity_stamps, disparity_stamp)
    lidar_stamp = sync_data(lidar_stamps, disparity_stamps, disparity_stamp)

    # find corresponding indices
    rgb_index = np.where(rgb_stamps == rgb_stamp)[0][0]
    lidar_index = np.where(lidar_stamps == lidar_stamp)[0][0]

    # images
    disp_index = disp_index+1
    rgb_index = rgb_index+1

    disparity_image_path = os.path.join(disp_path, f"disparity{dataset_num}_{disp_index}.png")
    rgb_image_path = os.path.join(rgb_path, f"rgb{dataset_num}_{rgb_index}.png")
    
    assert os.path.exists(disparity_image_path)
    assert os.path.exists(rgb_image_path)

    x, y, z, imc, imd, rgbu, rgbv, valid = coord3d(disparity_image_path, rgb_image_path)

    # optical frame to camera frame
    camera_coords = np.stack([z[valid], -x[valid], -y[valid]], axis=1)
    camera_coords = camera_coords.reshape(-1, 3)

    # homegeneous coordinates
    camera_coords = np.concatenate([camera_coords, np.ones((camera_coords.shape[0], 1))], axis=1)


    # camera frame to body frame
    body_coords = body_T_optical() @ camera_coords.T
    body_coords = body_coords.T

    # body frame to world frame
    x, y, theta = robot_poses[lidar_index]
    yaw = theta
    R_yaw = np.array([
        [np.cos(yaw), -np.sin(yaw), 0],
        [np.sin(yaw), np.cos(yaw), 0],
        [0, 0, 1]
    ])
    pose = np.eye(4)
    pose[:3, :3] = R_yaw
    pose[:3, 3] = np.array([x, y, 0])


    world_coords = pose @ body_coords.T
    world_coords = world_coords.T
    world_coords = world_coords[:, :3]


    colors = imc[rgbv[valid].astype(int),rgbu[valid].astype(int)]
    colors = colors.reshape(-1, 3)



(480, 640, 3)
